#### Ik heb '7 - Percentage of firms owned by women  by size_data.csv' geskipt aangezien het een leeg bestand is

In [23]:
import pandas as pd
import numpy as np
import glob
columns = ['Country', 'Year', 'measure_value', 'Indicator', 'Indicator_category', 'unit_of_measure', 'Source']

## Model retrieval

In [24]:
geos = pd.read_excel('../geo_locations_zoom.xlsx')
countries = geos.Name.tolist()
iso2 = geos.ISO2.tolist()
iso3 = geos.ISO2.tolist()

## Functions to clear transform data

In [25]:
def replaceCountries(df):
    df = df[~df['Country'].isna()]
    df.Country = df.Country.str.lower()
    df.Country.replace(['congo', 'democratic republic of the congo', 'united republic of tanzania', 'north macedonia', 
                        'state of palestine', 'viet nam', "côte d'ivoire", "democratic people's republic of korea", 
                        'republic of korea', 'swaziland', 'republic of moldova', 'china  hong kong special administrative region', 
                        'runion', 'china  macao special administrative region', 'british virgin islands', 'faeroe islands', 
                        'saint helena', 'united states virgin islands', 'wallis and futuna islands'], 
                       ['republic of congo', 'congo (democratic republic of the)', 'tanzania', 'macedonia', 
                        'palestine', 'vietnam', 'ivory coast', 'north korea', 
                        'south korea', 'eswatini', 'moldova', 'hong kong', 
                        'réunion', 'macao', 'virgin islands (british)', 'faroe islands', 
                        'saint helena, ascension and tristan da cunha', 'virgin islands (u.s.)', 'wallis and futuna'], inplace=True)
    return df

def setValue(df, newDf):
    if 'Value' in df.columns:
        newDf.measure_value = df.Value
    else:
        newDf.measure_value = df['Measure Value']
    return newDf

def setIndicatorCategory(file, df, newDf):
    if 'Sex' in df.columns:
        if 'Age' in df.columns and df.Age.isna().sum() == 0:
            newDf.Indicator_category = df.Sex + ' - ' + df.Age
        else:
            newDf.Indicator_category = df.Sex
    elif 'Age' in df.columns:
        newDf.Indicator_category = df.Age
    elif 'Measure Name' in df.columns:
        newDf.Indicator_category = df['Measure Name']
        newDf.unit_of_measure = 'Text'
    else:
        print("Can't find Indicator Category in file", file)
        df.columns
    newDf.Indicator_category.replace('not applicable', 'Whole population', inplace=True)
    return newDf

def setYearColumn(df, newDf):
    if 'Year' in df.columns:
        newDf.Year = df.Year
    else:
        newDf.Year = 2019
    return newDf

## Create DataFrame

In [26]:
dataframe = pd.DataFrame(columns=columns)
skip = ['7 - Percentage of firms owned by women  by size_data.csv']
files = glob.glob('*.csv')
for file in files:
    if file in skip:
        continue
    df = pd.read_csv(file, skiprows=2, na_values=['–','—', '-'])
    try:
        df = replaceCountries(df)
        newDf = pd.DataFrame(columns=columns)
        newDf.Country = df.Country
        newDf.Indicator = df['Indicator Name']
        newDf.unit_of_measure = 'Number'
        newDf.Source = 'Genderstats'
        
        newDf = setValue(df, newDf)
        newDf = setIndicatorCategory(file, df, newDf)
        newDf = setYearColumn(df, newDf)
        
        dataframe = pd.concat([dataframe, newDf])
    except Exception as e:
        print('Error in File: ', file)
        print(e)

## Remove rows with NaNs and non-numerical

In [27]:
dataframe.dropna(inplace=True)
dataframe[dataframe.isnull().any(axis=1)]

,Country,Year,measure_value,Indicator,Indicator_category,unit_of_measure,Source


In [28]:
#remove non-numerical values
dataframe['measure_value'] = pd.to_numeric(dataframe['measure_value'], errors = 'coerce')
dataframe = dataframe.dropna(subset = ['measure_value'])

### Align country names with ISO

In [29]:
country_iso_dict = dict(zip(countries, iso3))

dataframe['iso3'] = dataframe.Country.map(country_iso_dict)

In [30]:
dataframe['Indicator_short'] = dataframe['Indicator'].str[:150]

In [31]:
len_list = [len(x) for x in list(dataframe['Indicator_short'])]
max(len_list)

150

In [33]:
len(dataframe.Indicator_short.value_counts())

31

In [47]:
dataframe.Indicator_short.value_counts()

Number of new HIV infections per 1 000 uninfected population  by sex  age and key populations                                                             28649
Unemployment rate  by sex  age and persons with disabilities                                                                                              13839
Under-five mortality rate  by sex                                                                                                                         10507
Gross enrolment ratio in secondary education  by sex                                                                                                       8705
Primary education completion rate (proxy)  by sex                                                                                                          7510
Labour force participation rate for persons aged 15+  by sex                                                                                               7504
Adjusted net enrolment rate in primary e

In [38]:
df_17 = dataframe[dataframe['Year'] == 2017]


In [46]:
df_17[df_17['Indicator_short'].str.contains('proportion of ')]

,Country,Year,measure_value,Indicator,Indicator_category,unit_of_measure,Source,iso3,Indicator_short


In [50]:
df_17[df_17['Indicator_short'] == 'Proportion of births attended by skilled health personnel']

,Country,Year,measure_value,Indicator,Indicator_category,unit_of_measure,Source,iso3,Indicator_short
1120,burundi,2017,85.1,Proportion of births attended by skilled healt...,Female,Number,Genderstats,bi,Proportion of births attended by skilled healt...
1303,nigeria,2017,43.0,Proportion of births attended by skilled healt...,Female,Number,Genderstats,ng,Proportion of births attended by skilled healt...
1725,haiti,2017,41.7,Proportion of births attended by skilled healt...,Female,Number,Genderstats,ht,Proportion of births attended by skilled healt...


In [52]:
df_17.Indicator_short.value_counts()

Number of new HIV infections per 1 000 uninfected population  by sex  age and key populations                                                             1602
Unemployment rate  by sex  age and persons with disabilities                                                                                               768
Under-five mortality rate  by sex                                                                                                                          583
Proportion of adults (15 years and older) with an account at a bank or other financial institution or with a mobile-money-service provider  by sex         424
Gender gap in wages  by occupation  age and persons with disabilities                                                                                      292
Access to anti-retroviral drug  by sex                                                                                                                     268
Labour force participation rate for persons ag

In [14]:
# dataframe[dataframe['Indicator'] == 'Legal minimum age at marriage by sex']

In [15]:
# ## remove categorical text
# dataframe = dataframe[dataframe['Indicator'] != 'Legal minimum age at marriage by sex']
# dataframe = dataframe[dataframe['Indicator'] != 'Existence of laws on domestic violence']

## Save Dataframe as CSV

In [16]:
dataframe.to_csv('../genderstats_03-09-2019.csv', sep=',', encoding = 'UTF-8', index = False)